In [172]:
import sys
sys.path.append('/Users/celia/GitHub/mouse_bandit/data_preprocessing_code')
sys.path.append('/Users/celia/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing as bp
import sys
import os
import behavior_database

In [190]:
record_df = pd.read_csv('/Users/celia/GitHub/mouse_bandit/session_record.csv',index_col=0)
replace_row_shay = False # change to True if want to substitute old row with new version for imaging data
celia_session_record = True

### First time we ran this, had to add 'shay' to the old data base and add a new column for Markov
    record_df.insert(0,'Owner','shay')
    record_df.insert(5,'Markov', 0)

In [191]:
if record_df.columns[0] != 'Owner':
    record_df.insert(0,'Owner','shay') # add column for name of user
if record_df.columns[5] != 'Markov':
    record_df.insert(5,'Markov', 0) # add column for binary variable for Markov version of task. All previous values 0.

In [192]:
print(record_df.shape)
record_df.tail(5)

(807, 18)


,Owner,Session ID,Mouse ID,Date,Phase,Markov,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
802,shay,03102017_K13,K13,2017-03-10 00:00:00,2.0,0,0.8,0.2,50.0,50.0,364.0,4.0,247.0,0.84,2.0,1.0,35.0,35.0
803,shay,03102017_K7,K7,2017-03-10 00:00:00,2.0,0,0.8,0.2,50.0,50.0,358.0,4.0,250.0,0.84,2.0,1.0,35.0,35.0
804,shay,03102017_K9,K9,2017-03-10 00:00:00,2.0,0,0.9,0.1,50.0,50.0,329.0,5.0,249.0,0.79,2.0,1.0,35.0,35.0
805,shay,03102017_Q43,Q43,2017-03-10 00:00:00,2.0,0,0.9,0.1,50.0,50.0,556.0,9.0,476.0,0.90,2.0,1.0,35.0,35.0
806,shay,03102017_Q45,Q45,2017-03-10 00:00:00,2.0,0,0.1,0.9,50.0,50.0,642.0,10.0,509.0,0.88,2.0,1.0,35.0,35.0


## Add single day (multi mice) to celia_session_record

In [193]:
if celia_session_record == True:
    record_df = record_df.drop(record_df.index[:]) # if want to reset and start with (for new session record)
    
    MOUSE = ['Baby','Mustard','Raspberry','Sky','Wine']
    ROOT = '/Users/celia/Dropbox (HMS)/Celia_BehaviorFiles/'
    root_mouse={}
    for mouse in MOUSE: # make list of all mouse directories
        temp_dict = {mouse:os.path.join(ROOT,mouse)}
        root_mouse.update(temp_dict)

    date_folders = os.listdir(root_mouse[MOUSE[0]]) # assuming all mice have same dates 
    date_folders = date_folders[1:]
    for mouse in MOUSE:
        for date in date_folders:
            curr_dir = os.path.join(root_mouse[mouse], date)
            record_df = behavior_database.add_session(curr_dir, record_df, 'celia')
    record_df.to_csv(path_or_buf='/Users/celia/GitHub/mouse_bandit/celia_session_record.csv')

In [195]:
record_df.shape

(196, 18)

## Add any of Shay's data or change existing rows to contain imaging column

In [147]:
# any of Shay's old data to be loaded in (dates first)
DAY = '03102017'
ROOT = '/Users/celia/Dropbox (HMS)/Shay_BehaviorFiles/BehaviorData/datesfirst'
root_day = os.path.join(ROOT,DAY)

mouse_folders = os.listdir(root_day)

for mouse in mouse_folders:
    if not mouse[0] == '.':
        record_df = record_df[record_df['Session ID']!='%s_%s' % (DAY, mouse)] # remove the original row
        curr_dir = os.path.join(root_day,mouse)
        record_df = behavior_database.add_session(curr_dir,record_df,'shay', True)

if replace_row_shay == True:
    record_df.to_csv(path_or_buf='/Users/celia/GitHub/mouse_bandit/session_record.csv')

In [ ]:
record_df[record_df['Session ID']==session]

In [68]:
# any of Shay's old data to be loaded in (mouse name first)
MOUSE = 'Q43'
ROOT = '/Users/celia/Dropbox (HMS)/Shay_BehaviorFiles/BehaviorData/'
root_mouse = os.path.join(ROOT,MOUSE)

date_folders = os.listdir(root_mouse)

for date in date_folders:
    if not date[0] == '.':
        record_df = record_df[record_df['Session ID']!='%s_%s' % (date, MOUSE)] # remove the original row
        curr_dir = os.path.join(root_mouse,date)
        record_df = behavior_database.add_session(curr_dir,record_df,'shay', True)
if replace_row_shay == True:
    record_df.to_csv(path_or_buf='/Users/celia/GitHub/mouse_bandit/session_record.csv')

## Troubleshooting with the add_session function

In [94]:
'''set the input parameters to test'''
imaging=True
root_dir = '/Users/celia/Dropbox (HMS)/Shay_BehaviorFiles/BehaviorData/Q43/11172016'
owner_id='shay'

In [87]:
"""set up dataframe"""

output_df = record_df.copy()

names = [
    'Owner',
    'Session ID',
    'Mouse ID',
    'Date',
    'Phase',
    'Markov',
    'Left Reward Prob',
    'Right Reward Prob',
    'Block Range Min',
    'Block Range Max',
    'No. Trials',
    'No. Blocks',
    'No. Rewards',
    'p(high Port)',
    'Decision Window Duration',
    'Min Inter-trial-interval',
    'Left Solenoid Duration',
    'Right Solenoid Duration',
]

    
'''
load in trial data
'''
if imaging==True:
    columns = ['Elapsed Time (s)','Since last trial (s)',
    'Trial Duration (s)','Port Poked','Right Reward Prob',
    'Left Reward Prob','Reward Given', 'center_frame', 'decision_frame']
else:
    columns = ['Elapsed Time (s)','Since last trial (s)',
    'Trial Duration (s)','Port Poked','Right Reward Prob',
    'Left Reward Prob','Reward Given']

In [92]:
'''load in file'''

for file in os.listdir(root_dir):
    if not file[0] == '.':
        file_name = os.path.join(root_dir,file)
                
        if 'trials.csv' in file:
            trials = pd.read_csv(file_name,names=columns)
            session_id = file[:file.index('_',9)]
                    
        elif 'parameters.csv' in file:
            params = pd.read_csv(file_name)

    else:
        raise ValueError('In a hidden folder')          

In [101]:
# calculate p(high port)
high_p_port = np.zeros(trials.shape[0])
        
for row in trials.iterrows():
    i = row[0]
    current_trial = row[1]
            
    if ((current_trial['Right Reward Prob'] > current_trial['Left Reward Prob']) & (current_trial['Port Poked'] == 1)):
        high_p_port[i] = 1
    elif ((current_trial['Right Reward Prob'] < current_trial['Left Reward Prob']) & (current_trial['Port Poked'] == 2)):
        high_p_port[i] = 1
  
 # determine what stage in training we are in        
if params['centerPokeTrigger'].values == 0:
    phase = 0
elif (params['leftRewardProb'].values == params['rightRewardProb'].values):
    phase = 1
else:
    phase = 2
           
# convert date to datetime object
date = session_id[:8]
date_str =  np.str(date)
if (len(date_str) == 7):
    date_str = '0' + date_str
datetime = pd.to_datetime(date_str,format='%m%d%Y')
        
# check whether ismarkov is already a variable stored in params
if hasattr(params, 'ismarkov')==False:
    params['ismarkov']=0 # if not, set ismarkov to zero
        
# correct for block range stats if markov task was run
if params.loc[0,('ismarkov')]==1:
    params.loc[:,('blockRangeMin', 'blockRangeMax')] ='NaN'

            
# create a dictionary with all information
record = {
    'Owner': owner_id,
    'Session ID': session_id,
    'Mouse ID': session_id[9:],
    'Date': datetime,
    'Phase': phase,
    'Markov': params['ismarkov'],
    'Left Reward Prob': params['leftRewardProb'].values,
    'Right Reward Prob': params['rightRewardProb'].values,
    'Block Range Min': params['blockRangeMin'],
    'Block Range Max': params['blockRangeMax'],
    'No. Trials': trials.shape[0],
    'No. Blocks': np.sum(np.diff(trials['Right Reward Prob'].values) != 0),
    'No. Rewards': np.sum(trials['Reward Given']),
    'p(high Port)': np.round(high_p_port.mean(),decimals=2),
    'Decision Window Duration': params['centerPokeRewardWindow'],
    'Min Inter-trial-interval': params['minInterTrialInterval'],
    'Left Solenoid Duration': params['rewardDurationLeft'],
    'Right Solenoid Duration': params['rewardDurationRight']
                 }
        
#create DataFrame         
output_df = output_df.append(pd.DataFrame(data=record,columns=names),ignore_index=True)  
output_df = output_df.drop_duplicates()   
    